---
title: "Compte rendu du TP SVM"
format: html
author: "Fabian Condamy et Samy M'Rad"
jupyter: python3
---


L'objectif de ce TP est de se familiariser avec la méthode de classification dite de Support Vector Machine (SVM). Dans la suite, on implémentera cette technique sur différents jeux de données réels et simulés grâce au package scikit-learn de Python. On se concentrera principalement sur la compréhension et la maîtrise des principaux paramètres afin d’en ajuster la flexibilité et d’en évaluer l’impact sur les performances.


**Question 1**


In [ ]:
import sys
sys.path.append("scripts_python") # pour aller chercher la fonction de svm_source dans le sous dossier
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from svm_source import *

scaler = StandardScaler()

iris = datasets.load_iris()
X = iris.data
X = scaler.fit_transform(X)
y = iris.target
X = X[y != 0, :2]
y = y[y != 0]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

svm_linear = SVC(kernel='linear')
svm_linear.fit(X_train, y_train)
y_pred_linear = svm_linear.predict(X_test)
score_linear = svm_linear.score(X_test, y_test)
print('Score du modèle linéaire : %s' % score_linear)

def f_linear(xx):
    return svm_linear.predict(xx.reshape(1, -1))

plt.ion()
plt.figure(figsize=(15, 5))
plt.subplot(131)
plot_2d(X, y)
plt.title("iris dataset")

plt.subplot(132)
frontiere(f_linear, X, y)
plt.title("linear kernel")

Pour cet aléa précis, on trouve un score de 0,64, ce qui est faible. Comme on peut le voir sur le jeu de données iris, les deux classes sont bien mélangées, ce qui peut expliquer ce score médiocre. 


In [ ]:
# Définition des hyperparamètres à tester
parameters = {'kernel': ['linear'], 'C': list(np.logspace(-3, 3, 200))}

# Créer le modèle SVM (ici juste l'objet, le kernel sera défini par GridSearchCV)
svm = SVC()

# GridSearchCV pour trouver le meilleur C
svm_linear_opt = GridSearchCV(svm, parameters, cv=5)  # cv=5 : validation croisée 5 folds
svm_linear_opt.fit(X_train, y_train)  # entraîner sur le train set

print("Score généralisé pour le noyau linéaire : Train : %.3f | Test : %.3f" %
      (svm_linear_opt.score(X_train, y_train),
       svm_linear_opt.score(X_test, y_test)))

Pour le score généralisé, on trouve une performance de 0,707 pour la partie d'apprentissage et 0,680 pour la partie de test. Ainsi, même en essayant d'optimiser le noyau linéaire, la précision est faible.
On va maintenant voir si une méthode polynomiale peut améliorer ce score.

**Question 2**


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

svm_poly = SVC(kernel='poly')
svm_poly.fit(X_train, y_train)
y_pred_poly = svm_poly.predict(X_test)
score_poly = svm_poly.score(X_test, y_test)
print('Score du modèle linéaire : %s' % score_poly)

def f_poly(xx):
    return svm_poly.predict(xx.reshape(1, -1))

plt.ion()
plt.figure(figsize=(15, 5))
plt.subplot(131)
plot_2d(X, y)
plt.title("Jeu de données iris")

plt.subplot(132)
frontiere(f_poly, X, y)
plt.title("Noyau polynomial")

Pour un noyau polynomial et cet aléa, on trouve un score de 0,6, ce qui est encore plus faible que pour le noyau linéaire. Ce résultat peut sembler assez paradoxal de prime abord, mais il semble en réalité logique au vu de la structure des données, le modèle a dû certainement overfitter.


In [ ]:
Cs = list(np.logspace(-3, 3, 5))
gammas = 10. ** np.arange(-2, 2)  # gamma = [0.01, 0.1, 1, 10]
degrees = np.r_[1, 2, 3]

parameters = {'kernel': ['poly'], 'C': Cs, 'gamma': gammas, 'degree': degrees}

svm = SVC()

svm_poly_opt = GridSearchCV(svm, parameters, cv=5)
svm_poly_opt.fit(X_train, y_train)

print("Best parameters:", svm_poly_opt.best_params_)

print("Score généralisé pour le noyau polynomial : Train : %.3f | Test : %.3f" %
      (svm_poly_opt.score(X_train, y_train),
       svm_poly_opt.score(X_test, y_test)))

Pour les paramètres généralisés, on trouve un score de 0,707 pour la partie d'apprentissage et 0,64 pour le test. Ainsi, on a le même score généralisé que pour le noyau linéaire pour la partie apprentissage, mais lorsqu'on l'applique à la partie de test, on perd plus en précision que pour le modèle linéaire.

Ainsi, ces 2 classes du jeu de données iris semble très difficiles à séparer, la méthode de SVM est peu adaptée ici.

**Question 3 (facultative)**

Commençons par générer le jeu de données très déséquilibré :


In [ ]:
import sys
sys.path.append("scripts_python") # pour aller chercher la fonction de svm_source

import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from svm_source import *
# Création du jeu de données

n = 1000 # nombre d'observations
classes = np.random.choice([0,1], size=n, p=[0.9,0.1]) # classes 1 et 2 avec respectivement p=90% et p=10%

# Affichage pour vérifier
print(np.sort(classes))

# Variables pour SVM
X = np.random.randn(n, 2)
Y = classes

def plot_svm(C_value):
    clf = SVC(kernel='linear', C=C_value)
    clf.fit(X, Y)

    plt.figure()
    plt.title(f"SVM linéaire pour C={C_value}")

    # Tracer les points
    plt.scatter(X[Y == 0][:, 0], X[Y == 0][:, 1], label="Classe 1 (majoritaire)", alpha=0.5)
    plt.scatter(X[Y == 1][:, 0], X[Y == 1][:, 1], label="Classe 2 (minoritaire)", alpha=0.8)

    # Tracer la frontière
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xx, yy = np.meshgrid(
        np.linspace(xlim[0], xlim[1], 100),
        np.linspace(ylim[0], ylim[1], 100)
    )
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contour(xx, yy, Z, levels=[0], linewidths=2, linestyles="--", alpha=0.7)
    plt.legend()
    plt.show()

# Afficher plusieurs valeurs de C
for C in [1, 0.1, 0.01]:
    plot_svm(C)

Sur un jeu de données purement aléatoire, on observe que le paramètre C n'a aucune influence, les frontières n'apparaissent même pas. Ceci est logique car les données sont impossibles à séparer facilement, il n'y a aucune logique sous-jacente à cette répartition des points.

Si on prend maintenant un jeu de données plus structuré :


In [ ]:
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt

# Génération d'un dataset structuré (mais déséquilibré)
n_majoritaire = 90
n_minoritaire = 10

# Classe majoritaire centrée en (0,0)
X_majoritaire = np.random.randn(n_majoritaire, 2) * 0.8 + np.array([0, 0])
# Classe minoritaire centrée en (3,3)
X_minoritaire = np.random.randn(n_minoritaire, 2) * 0.8 + np.array([3, 3])

X = np.vstack((X_majoritaire, X_minoritaire))
Y = np.array([0]*n_majoritaire + [1]*n_minoritaire)

def plot_svm(C_value):
    clf = SVC(kernel='linear', C=C_value)
    clf.fit(X, Y)

    plt.figure()
    plt.title(f"SVM linéaire pour C={C_value}")

    # Tracer les points
    plt.scatter(X[Y == 0][:, 0], X[Y == 0][:, 1], label="Classe 1 (majoritaire)", alpha=0.5)
    plt.scatter(X[Y == 1][:, 0], X[Y == 1][:, 1], label="Classe 2 (minoritaire)", alpha=0.8)

    # Tracer la frontière
    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    xx, yy = np.meshgrid(
        np.linspace(xlim[0], xlim[1], 100),
        np.linspace(ylim[0], ylim[1], 100)
    )
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    ax.contour(xx, yy, Z, levels=[0], linewidths=2, linestyles="--", alpha=0.7)
    plt.legend()
    plt.show()

# Afficher plusieurs valeurs de C
for C in [1, 0.1, 0.01]:
    plot_svm(C)

Ici, on voit l'action du paramètre C, la frontière se déplace dans les différents exemples. 


**Question 4**


**Question 7**

Le biais introduit par le code se situe dans ces deux lignes :


In [ ]:
X -= np.mean(X, axis=0)
X /= np.std(X, axis=0)

En effet, ici on standardise les données avant de séparer notre jeu de données en 2 pour le train et le test, ce qui fait que l'on utilise à la fois les données du train et du test pour calculer la moyenne et l'écart-type qui vont ensuite être appliqués à l'ensemble de nos données pour les standardiser. Ceci crée un biais car les données de test ne doivent pas servir pour créer le modèle.